# Azure OpenAI Service - Chat on private data using LangChain

Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values:

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

Then, let's install all dependencies:

In [ ]:
!pip3 install langchain

In [11]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
import numpy as np

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
print("Did load from env file?:", load_dotenv('../.env'))

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = "azure"

# Init LLM and embeddings model
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0, openai_api_version="2023-03-15-preview")
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002", 
    openai_api_base=os.getenv('OPENAI_API_BASE'),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_version="2023-03-15-preview",
    openai_api_type="azure",
    chunk_size=1)

embed_vals = embeddings.embed_documents(["Hello, how are you?"])
print(f"got embeddings with shape {np.array(embed_vals).shape}")

Did load from env file?: True
got embeddings with shape (1, 1536)


First, we load up our documents from the `data` directory:

In [12]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from timeit import default_timer

start_time = default_timer()
loader = DirectoryLoader('../data/qna/', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
runtime = default_timer() - start_time
print(f"Number of documents: {len(docs)} in {runtime:.2f} seconds")
print(docs)

Number of documents: 6 in 0.01 seconds
[Document(page_content='\n# What is Azure OpenAI?\n\nThe Azure OpenAI service provides REST API access to OpenAI\'s powerful language models including the GPT-3, Codex and Embeddings model series. These models can be easily adapted to your specific task including but not limited to content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or our web-based interface in the Azure OpenAI Studio.\n\n### Features overview\n\n| Feature | Azure OpenAI |\n| --- | --- |\n| Models available | GPT-3 base series <br> Codex series <br> Embeddings series <br> Learn more in our [Models](./concepts/models.md) page.|\n| Fine-tuning | Ada <br> Babbage <br> Curie <br> Cushman* <br> Davinci* <br> \\* available by request. Please open a support request|\n| Price | [Available here](https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/) |\n| Virtual n

Next, let's ingest them into FAISS so we can efficiently query our embeddings:

In [13]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=docs, embedding=embeddings)

Now let's create a chain that can do the whole chat on our embedding database:

In [14]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

Now let's ask a question:

In [15]:
chat_history = []
query = "what is azure openai service?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

Azure OpenAI is a service that provides REST API access to OpenAI's language models, including GPT-3, Codex, and Embeddings model series. These models can be used for content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or a web-based interface in the Azure OpenAI Studio. The service is available in East US, South Central US, and West Europe regions. Additionally, Azure OpenAI offers private networking, regional availability, and responsible AI content filtering.


We can use this to easy implement chat conversations:

In [16]:
chat_history = []

query = "what is Azure OpenAI Service?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

chat_history = [(query, result["answer"])]
query = "Which regions does the service support?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

Question: what is Azure OpenAI Service?
Answer: Azure OpenAI Service is a cloud-based service that provides REST API access to OpenAI's language models, including GPT-3, Codex, and Embeddings model series. These models can be used for various tasks such as content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or a web-based interface in the Azure OpenAI Studio. The service is built on Microsoft Azure and offers features such as virtual network support, managed identity, and responsible AI content filtering. However, access to the service is currently limited due to high demand and Microsoft's commitment to responsible AI.
Question: Which regions does the service support?
Answer: The Azure OpenAI Service is currently available in the East US, South Central US, and West Europe regions.


In [22]:
#Now lets test if this is working without documents?
chat_history = []

#TODO: Fix this - define a list of text out of context.
texts =
db_no_docs = FAISS.from_texts(texts, embedding=embeddings)

qa_no_retriever = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db_no_docs.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)
query = "what is Azure OpenAI Service?"
result = qa_no_retriever({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])


Question: what is Azure OpenAI Service?
Answer: Azure OpenAI Service is a cloud-based service provided by Microsoft Azure that allows developers to access OpenAI's powerful artificial intelligence models and tools. It enables developers to build intelligent applications that can understand natural language, recognize images, and perform other complex tasks.
